In [1]:
import time
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy import sparse
from datetime import datetime
from sklearn import preprocessing
from scipy.stats import skew, boxcox,boxcox_normmax
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from bayes_opt import BayesianOptimization
from sklearn.metrics import log_loss

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


seed = 1234

/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Load Data

In [2]:
data_path = "../input/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
train_df = pd.read_json(train_file).reset_index()
test_df = pd.read_json(test_file).reset_index()
ntrain = train_df.shape[0]
print train_df.shape
print test_df.shape
print ntrain

(49352, 16)
(74659, 15)
49352


In [3]:
# sc_price
tmp = pd.concat([train_df['price'],test_df['price']])
ulimit = np.percentile(tmp.values, 99)
llimit = np.percentile(tmp.values, 1)

train_df.loc[:,'sc_price'] = train_df['price'].values.reshape(-1, 1)
test_df.loc[:,'sc_price'] = test_df['price'].values.reshape(-1, 1)

train_df.loc[train_df['sc_price']>ulimit, ['sc_price']] = ulimit
test_df.loc[test_df['sc_price']>ulimit, ['sc_price']] = ulimit
train_df.loc[train_df['sc_price']<llimit, ['sc_price']] = llimit
test_df.loc[test_df['sc_price']<llimit, ['sc_price']] = llimit



# sc_ba_price
inx_train = train_df['bathrooms'] == 0
inx_test = test_df['bathrooms'] == 0

non0_inx_train = ~inx_train
non0_inx_test = ~inx_test

train_df.loc[non0_inx_train,'sc_ba_price'] = train_df.loc[non0_inx_train,'sc_price']\
                                                /train_df.loc[non0_inx_train,'bathrooms']
test_df.loc[non0_inx_test,'sc_ba_price'] = test_df.loc[non0_inx_test,'sc_price']\
                                                /test_df.loc[non0_inx_test,'bathrooms']

train_df.loc[inx_train,'sc_ba_price'] = 0
test_df.loc[inx_test,'sc_ba_price'] = 0

train_df.loc[non0_inx_train,'bathrooms0'] = 1
test_df.loc[non0_inx_test,'bathrooms0'] = 1

train_df.loc[inx_train,'bathrooms0'] = 0
test_df.loc[inx_test,'bathrooms0'] = 0

# price per bedrooms

inx_train = train_df['bedrooms'] == 0
inx_test = test_df['bedrooms'] == 0

non0_inx_train = ~inx_train
non0_inx_test = ~inx_test

train_df.loc[non0_inx_train,'sc_be_price'] = train_df.loc[non0_inx_train,'sc_price'] \
                                                /train_df.loc[non0_inx_train,'bedrooms']
test_df.loc[non0_inx_test,'sc_be_price'] = test_df.loc[non0_inx_test,'sc_price']\
                                                /test_df.loc[non0_inx_test,'bedrooms']

train_df.loc[inx_train,'sc_be_price'] = 0
test_df.loc[inx_test,'sc_be_price'] = 0

train_df.loc[non0_inx_train,'bedrooms0'] = 1
test_df.loc[non0_inx_test,'bedrooms0'] = 1

train_df.loc[inx_train,'bedrooms0'] = 0
test_df.loc[inx_test,'bedrooms0'] = 0
# bathrooms

ulimit = 5

train_df['sc_bathrooms']=train_df['bathrooms']
test_df['sc_bathrooms']=test_df['bathrooms']

train_df.loc[train_df['sc_bathrooms']>ulimit,['sc_bathrooms']] = ulimit
test_df.loc[test_df['sc_bathrooms']>ulimit,['sc_bathrooms']] = ulimit

# bedrooms

ulimit = 8

train_df['sc_bedrooms']=train_df['bedrooms']
test_df['sc_bedrooms']=test_df['bedrooms']

train_df.loc[train_df['sc_bedrooms']>ulimit, ['sc_bedrooms']] = ulimit
test_df.loc[test_df['sc_bedrooms']>ulimit,['sc_bedrooms']] = ulimit

# longitude

tmp = pd.concat([train_df['longitude'],test_df['longitude']])
llimit = np.percentile(tmp.values, 0.1)
ulimit = np.percentile(tmp.values, 99.9)

train_df['sc_longitude']=train_df['longitude']
test_df['sc_longitude']=test_df['longitude']

train_df.loc[train_df['sc_longitude']>ulimit, ['sc_longitude']] = ulimit
test_df.loc[test_df['sc_longitude']>ulimit, ['sc_longitude']] = ulimit
train_df.loc[train_df['sc_longitude']<llimit, ['sc_longitude']] = llimit
test_df.loc[test_df['sc_longitude']<llimit, ['sc_longitude']] = llimit

# latitude

tmp = pd.concat([train_df['latitude'],test_df['latitude']])
llimit = np.percentile(tmp.values, 0.1)
ulimit = np.percentile(tmp.values, 99.9)

train_df['sc_latitude']=train_df['latitude']
test_df['sc_latitude']=test_df['latitude']

train_df.loc[train_df['sc_latitude']>ulimit, ['sc_latitude']] = ulimit
test_df.loc[test_df['sc_latitude']>ulimit, ['sc_latitude']] = ulimit
train_df.loc[train_df['sc_latitude']<llimit, ['sc_latitude']] = llimit
test_df.loc[test_df['sc_latitude']<llimit, ['sc_latitude']] = llimit


features_to_use  = ["sc_bathrooms", "sc_bedrooms", "sc_latitude", "sc_longitude",
                    "sc_price", "sc_ba_price", "sc_be_price"]

In [4]:
# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

# adding all these new features to use list #
features_to_use.extend(["num_photos", "num_features", "num_description_words", "created_month", 
                        "created_day", "created_hour"])


In [5]:
full_data=pd.concat([train_df,test_df])

# SSL = preprocessing.StandardScaler()
# for col in features_to_use:
#     full_data[col], lam = boxcox(full_data[col] - full_data[col].min() + 1)
#     full_data[col] = SSL.fit_transform(full_data[col].values.reshape(-1,1)) 
skewed_cols = full_data[features_to_use].apply(lambda x: skew(x.dropna()))

SSL = preprocessing.StandardScaler()
skewed_cols = skewed_cols[skewed_cols > 0.25].index.values
for skewed_col in skewed_cols:
    full_data[skewed_col], lam = boxcox(full_data[skewed_col] - full_data[skewed_col].min() + 1)
    print skewed_col, '\t', lam
for col in features_to_use:
    full_data[col] = SSL.fit_transform(full_data[col].values.reshape(-1,1))
    train_df[col] = full_data.iloc[:ntrain][col]
    test_df[col] = full_data.iloc[ntrain:][col]

    
del full_data

sc_bathrooms 	-0.974792935849
sc_bedrooms 	0.474243433469
sc_longitude 	-12.8479106804
sc_price 	0.34122013083
sc_ba_price 	0.668968094532
sc_be_price 	0.462918736223
num_photos 	0.475560723219
num_features 	0.359523382085
num_description_words 	0.543711145224
created_hour 	-0.389913999668


/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [6]:
# fig = train_df[features_to_use].hist(bins=100)

In [7]:
# fig = test_df[features_to_use].hist(bins=100)

In [8]:
# test_df.sc_longitude.hist(bins=100)

In [9]:
features_to_use.extend(["listing_id","bedrooms0",'bathrooms0'])
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)

In [10]:
train_df['features'] = train_df["features"]\
                        .apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))\
                        .apply(lambda x: x.lower())
test_df['features'] = test_df["features"]\
                        .apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))\
                        .apply(lambda x: x.lower())

print(train_df["features"].head())
tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

sparse_features = tfidf.get_feature_names()

0                                                     
1    doorman elevator fitness_center cats_allowed d...
2    laundry_in_building dishwasher hardwood_floors...
3                               hardwood_floors no_fee
4                                              pre-war
Name: features, dtype: object


In [11]:
train_X = sparse.hstack([train_df[features_to_use], tr_sparse]).tocsr()
test_X = sparse.hstack([test_df[features_to_use], te_sparse]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
weight_num_map = {'high':1, 'medium':1, 'low':1}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))
W_train = np.array(train_df['interest_level'].apply(lambda x: weight_num_map[x]))

all_features = features_to_use + sparse_features
print train_X.shape, test_X.shape

(49352, 220) (74659, 220)


In [9]:
# cv_dataset = xgb.DMatrix(train_X, label = train_y,
#                          weight = W_train,
#                          feature_names = all_features)

In [10]:
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, train_size=.90, random_state=1234)
print X_train.shape
print X_val.shape
# xgtrain = xgb.DMatrix(X_train, label=y_train)

(44416, 218)
(4936, 218)


In [11]:
# rgr = xgb.XGBClassifier(objective = 'multi:softprob',
#                        learning_rate = 0.1,
#                        n_estimators = 10000,
#                        nthread = -1)

# rgr.fit(X_train,y_train,
#         eval_set=[(X_val,y_val)],
#         eval_metric='mlogloss',
# #         num_class = 3,
#         early_stopping_rounds=20,
#         verbose=False
#        )

# tmp1 = rgr.predict_proba(test_X)
# tmp1=pd.DataFrame(tmp1)
# tmp1.columns = ['high','medium','low']
# plt.scatter(range(74659),tmp1.high)

In [28]:
learning_rate = 0.1
best_score = 1000
train_param = 0
for x in [3,4,5,6,7,8,9,10]:
    rgr = xgb.XGBClassifier(
        objective='multi:softprob',
        seed = 1234, # use a fixed seed during tuning so we can reproduce the results
        learning_rate = learning_rate,
        n_estimators = 10000,
        max_depth= x,
        nthread = -1,
        silent = False
    )
    rgr.fit(
        X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric='mlogloss',
        early_stopping_rounds=20,
        verbose=False
    )
    
    if rgr.best_score < best_score:
        best_score = rgr.best_score
        train_param = x

    print x, '\t', rgr.best_score

3 	0.556399
4 	0.556165
5 	0.553328
6 	0.552869
7 	0.557626
8 	0.556641
9 	0.559613
10 	0.562173


In [29]:
max_depth = train_param
print train_param
# 3 	0.550966
# 4 	0.546335
# 5 	0.546948
# 6 	0.548233
# 7 	0.548868
# 8 	0.551074
# 9 	0.553485
# 10 	0.557234

In [31]:
best_score = 1000
train_param = 0
for x in [5,10,20,50,80,120,180,240,300]:
    rgr = xgb.XGBClassifier(
        objective='multi:softprob',
        seed = 1234, # use a fixed seed during tuning so we can reproduce the results
        learning_rate = learning_rate,
        n_estimators = 10000,
        max_depth= max_depth,
        nthread = -1,
        silent = False,
        min_child_weight = x
    )
    rgr.fit(
        X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric='mlogloss',
        early_stopping_rounds=20,
        verbose=False
    )
    
    if rgr.best_score < best_score:
        best_score = rgr.best_score
        train_param = x
        

    print x, '\t', rgr.best_score

5 	0.552672
10 	0.552702
20 	0.554569
50 	0.555714
80 	0.558212
120 	0.556721
180 	0.556979
240 	0.563283
300 	0.565675


In [33]:
min_child_weight = train_param
print min_child_weight

5


In [34]:
best_score = 1000
train_param = 0
for x in [0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    rgr = xgb.XGBClassifier(
        objective='multi:softprob',
        seed = 1234, # use a fixed seed during tuning so we can reproduce the results
        learning_rate = learning_rate,
        n_estimators = 10000,
        max_depth= max_depth,
        nthread = -1,
        silent = False,
        min_child_weight = min_child_weight,
        colsample_bytree = x
    )
    rgr.fit(
        X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric='mlogloss',
        early_stopping_rounds=20,
        verbose=False
    )

    if rgr.best_score < best_score:
        best_score = rgr.best_score
        train_param = x
        

    print x, '\t', rgr.best_score

0.3 	0.544995
0.4 	0.548923
0.5 	0.547074
0.6 	0.546579
0.7 	0.551113
0.8 	0.550493
0.9 	0.551921


In [37]:
colsample_bytree = 0.6
print train_param

0.3


In [38]:
best_score = 1000
train_param = 0
for x in [0.5,0.6,0.7,0.8,0.9]:
    rgr = xgb.XGBClassifier(
        objective='multi:softprob',
        seed = 1234, # use a fixed seed during tuning so we can reproduce the results
        learning_rate = learning_rate,
        n_estimators = 10000,
        max_depth= max_depth,
        nthread = -1,
        silent = False,
        min_child_weight = min_child_weight,
        colsample_bytree = colsample_bytree,
        subsample = x
    )
    rgr.fit(
        X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric='mlogloss',
        early_stopping_rounds=20,
        verbose=False
    )
    if rgr.best_score < best_score:
        best_score = rgr.best_score
        train_param = x
        

    print x, '\t', rgr.best_score

0.5 	0.551334
0.6 	0.55245
0.7 	0.54911
0.8 	0.550137
0.9 	0.548913


In [39]:
subsample = train_param
print train_param

0.9


In [40]:
best_score = 1000
train_param = 0
for x in [0, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3.0]:
    rgr = xgb.XGBClassifier(
        objective='multi:softprob',
        seed = 1234, # use a fixed seed during tuning so we can reproduce the results
        learning_rate = learning_rate,
        n_estimators = 10000,
        max_depth= max_depth,
        nthread = -1,
        silent = False,
        min_child_weight = min_child_weight,
        colsample_bytree = colsample_bytree,
        subsample = subsample,
        gamma = x
    )
    rgr.fit(
        X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric='mlogloss',
        early_stopping_rounds=20,
        verbose=False
    )

    if rgr.best_score < best_score:
        best_score = rgr.best_score
        train_param = x
        

    print x, '\t', rgr.best_score

0 	0.548913
0.3 	0.549075
0.6 	0.547017
0.9 	0.546771
1.2 	0.549405
1.5 	0.546167
1.8 	0.548024
2.1 	0.548317
2.4 	0.546834
2.7 	0.549393
3.0 	0.545459


In [41]:
gamma = train_param
print train_param

3.0


In [42]:
xgtrain = xgb.DMatrix(train_X, label=train_y) 

def xgb_evaluate(min_child_weight, colsample_bytree, max_depth, subsample, gamma):
    params = dict()
    params['objective']='multi:softprob'
    params['eval_metric']='mlogloss',
    params['num_class']=3
    params['silent']=1
    params['eta'] = 0.1
    params['verbose_eval'] = True
    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    
    cv_result = xgb.cv(
        params, xgtrain, 
        num_boost_round=10000, nfold=10,
        metrics = 'mlogloss',
        seed=seed,callbacks=[xgb.callback.early_stop(50)]
    )
    
    return -cv_result['test-mlogloss-mean'].values[-1]


xgb_BO = BayesianOptimization(
    xgb_evaluate, 
    {
        'max_depth': (3,6),
        'min_child_weight': (1,50),
        'colsample_bytree': (0.7,1),
        'subsample': (0.7,1),
        'gamma': (0.3,2.1)
    }
)

xgb_BO.maximize(init_points=10, n_iter=40)

Initialization
---------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[1093]	train-mlogloss:0.401341+0.00132894	test-mlogloss:0.542346+0.00573164

    1 | 08m25s |   -0.54235 |             0.8668 |    1.9502 |      4.7133 |             2.6959 |      0.8435 | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[566]	train-mlogloss:0.420788+0.00165193	test-mlogloss:0.547142+0.00599595

    2 | 05m31s |   -0.54714 |             0.8646 |    0.3752 |      5.0573 |            39.3628 |      0.7776 | 
Multiple eval metrics have 

/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
/home/xujin/AI/anaconda2/lib/python2.7/site-packages/bayes_opt/helpers.py:95: RuntimeWarning: divide by zero encountered in true_divide
  z = (mean - y_max - xi)/std


   11 | 07m58s |   -0.54468 |             0.9993 |    0.5129 |      4.0034 |             2.0828 |      0.8933 | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[1153]	train-mlogloss:0.407059+0.00179506	test-mlogloss:0.54332+0.00567749



/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -1.30528409e-05]), 'nit': 5, 'funcalls': 53}
  " state: %s" % convergence_dict)
/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00028024]), 'nit': 4, 'funcalls': 47}
  " state: %s" % convergence_dict)
/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([  1.26466873e-05]), 'nit': 6, 'funcalls': 49}
  " state: %s" % convergence_dict)


   12 | 08m23s |   -0.54332 |             0.7552 |    2.0722 |      4.9775 |             7.0850 |      0.8323 | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[587]	train-mlogloss:0.377684+0.00163147	test-mlogloss:0.544077+0.00571811

   13 | 05m32s |   -0.54408 |             0.7474 |    1.3964 |      5.3378 |             2.3705 |      0.7276 | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[2063]	train-mlogloss:0.420522+0.00145732	test-mlogloss:0.54445+0.00571767



...

   46 | 05m30s |   -0.54727 |             0.7354 |    1.4790 |      5.6728 |            34.0076 |      0.7098 | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[788]	train-mlogloss:0.416902+0.00171272	test-mlogloss:0.546026+0.00653838



/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00232891]), 'nit': 3, 'funcalls': 48}
  " state: %s" % convergence_dict)
/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 0.00010399]), 'nit': 7, 'funcalls': 65}
  " state: %s" % convergence_dict)


   47 | 08m04s |   -0.54603 |             0.7913 |    2.0545 |      5.5018 |            39.8571 |      0.8385 | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[1046]	train-mlogloss:0.425592+0.00155637	test-mlogloss:0.546166+0.00545797

   48 | 11m04s |   -0.54617 |             0.7892 |    2.0132 |      4.8927 |            26.4855 |      0.7682 | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[629]	train-mlogloss:0.390545+0.00148152	test-mlogloss:0.545658+0.00690329



/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00094195]), 'nit': 3, 'funcalls': 48}
  " state: %s" % convergence_dict)


   49 | 09m36s |   -0.54566 |             0.9852 |    0.3648 |      5.9029 |            22.6354 |      0.8157 | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[897]	train-mlogloss:0.420927+0.00130185	test-mlogloss:0.546387+0.00584617

   50 | 07m26s |   -0.54639 |             0.8990 |    0.4912 |      4.2103 |            30.6949 |      0.8513 | 


In [43]:
xgb_bo_scores = pd.DataFrame([[s[0]['max_depth'],
                               s[0]['min_child_weight'],
                               s[0]['colsample_bytree'],
                               s[0]['subsample'],
                               s[0]['gamma'],
                               s[1]] for s in zip(xgb_BO.res['all']['params'],xgb_BO.res['all']['values'])],
                            columns = ['max_depth',
                                       'min_child_weight',
                                       'colsample_bytree',
                                       'subsample',
                                       'gamma',
                                       'score'])
xgb_bo_scores=xgb_bo_scores.sort_values('score',ascending=False)
xgb_bo_scores.head(10)

,max_depth,min_child_weight,colsample_bytree,subsample,gamma,score
1,4.977517,7.085047,0.755216,0.832299,2.072250,-0.543320
12,5.294358,6.430458,0.778670,0.794628,2.053074,-0.543492
10,4.831209,1.988818,0.976038,0.791368,1.601972,-0.543742
2,5.337825,2.370541,0.747375,0.727643,1.396417,-0.544077
3,3.844964,2.067151,0.828660,0.821156,1.951036,-0.544450
0,4.003396,2.082787,0.999265,0.893346,0.512947,-0.544678
25,4.607936,5.782687,0.904582,0.933866,0.366536,-0.545158
17,5.319597,25.284915,0.932358,0.949350,2.065272,-0.545291
13,4.126909,16.160868,0.857025,0.913964,0.708522,-0.545368
21,3.624415,8.166120,0.709067,0.845712,1.583732,-0.545375


In [12]:
def xgb_blend(estimators, train_x, train_y, test_x, fold, early_stopping_rounds=0):
    N_params = len(estimators)
    print ("Blend %d estimators for %d folds" % (N_params, fold))
    skf = KFold(n_splits=fold,random_state=seed)
    N_class = len(set(train_y))
        
    train_blend_x = np.zeros((train_x.shape[0], N_class*N_params))
    test_blend_x = np.zeros((test_x.shape[0], N_class*N_params))
    scores = np.zeros ((fold,N_params))
    best_rounds = np.zeros ((fold, N_params))
    
    for j, est in enumerate(estimators):
        est.set_params(objective = 'multi:softprob')
        est.set_params(silent = False)
        est.set_params(learning_rate = 0.03)
        est.set_params(n_estimators=100000)
        
        print ("Model %d: %s" %(j+1, est))

        test_blend_x_j = np.zeros((test_x.shape[0], N_class*fold))
    
        for i, (train_index, val_index) in enumerate(skf.split(train_x)):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x[train_index]
            train_y_fold = train_y[train_index]
            val_x_fold = train_x[val_index]
            val_y_fold = train_y[val_index]   

            est.fit(train_x_fold,train_y_fold,
                    eval_set = [(val_x_fold, val_y_fold)],
                    eval_metric = 'mlogloss',
                    early_stopping_rounds=early_stopping_rounds,
                    verbose=False)
            best_round=est.best_iteration
            best_rounds[i,j]=best_round
            print ("best round %d" % (best_round))
            val_y_predict_fold = est.predict_proba(val_x_fold,ntree_limit=best_round)
            score = log_loss(val_y_fold, val_y_predict_fold)
            print ("Score: ", score)
            scores[i,j]=score
            train_blend_x[val_index, (j*N_class):(j+1)*N_class] = val_y_predict_fold
            
            test_blend_x_j[:,(i*N_class):(i+1)*N_class] = est.predict_proba(test_x,ntree_limit=best_round)
            print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))
            
        test_blend_x[:,(j*N_class):(j+1)*N_class] = \
                np.stack([test_blend_x_j[:,range(0,N_class*fold,N_class)].mean(1),
                          test_blend_x_j[:,range(1,N_class*fold,N_class)].mean(1),
                          test_blend_x_j[:,range(2,N_class*fold,N_class)].mean(1)]).T
        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x, scores,best_rounds,test_blend_x_j)


In [ ]:
estimators = [xgb.XGBClassifier(max_depth = 6,
                              min_child_weight = 5,
                              colsample_bytree = 0.6,
                              subsample = 0.9,
                              gamma = 3),
#              xgb.XGBClassifier(max_depth = 5,
#                               min_child_weight = 6,
#                               colsample_bytree = 0.778670,
#                               subsample = 0.794628,
#                               gamma = 2.053074)
             ]





# xgb_params = [{'max_depth':6,
#                'min_child_weight':5,
#                'colsample_bytree':0.6,
#                'subsample':0.9,
#                'gamma':3},
# #               score -0.543320        
#               {'max_depth':5,
#                'min_child_weight':6,
#                'colsample_bytree':0.778670,
#                'subsample':0.794628,
#                'gamma':2.053074},
#               score -0.543492
#               {'max_depth':4,
#                'min_child_weight':1,
#                'colsample_bytree':0.976038,
#                'subsample':0.791368,
#                'gamma':1.601972},
# #               score -0.543742  
#               {'max_depth':5,
#                'min_child_weight':2,
#                'colsample_bytree':0.747375,
#                'subsample':0.727643,
#                'gamma':1.396417},
# #               score -0.544077      
#               {'max_depth':3,
#                'min_child_weight':2,
#                'colsample_bytree':0.828660,
#                'subsample':0.821156,
#                'gamma':1.951036}
# #               score -0.544450
#              ]

(train_blend_x_xgb,
 test_blend_x_xgb,
 blend_scores_xgb,
 best_rounds_xgb,tmp) = xgb_blend(estimators,
                              train_X,train_y,
                              test_X,
                              10,
                              300)

# print (np.mean(blend_scores_xgb_le,axis=0))
# print (np.mean(best_rounds_xgb_le,axis=0))

Blend 1 estimators for 10 folds
Model 1: XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.6,
       gamma=3, learning_rate=0.03, max_delta_step=0, max_depth=6,
       min_child_weight=5, missing=None, n_estimators=100000, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=0.9)
Model 1 fold 1


In [22]:
now = datetime.now()

name_train_blend = '../output/train_blend_xgb_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend = '../output/test_blend_xgb_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'



print (np.mean(blend_scores_xgb,axis=0))
print (np.mean(best_rounds_xgb,axis=0))
np.savetxt(name_train_blend,train_blend_x_xgb, delimiter=",")
np.savetxt(name_test_blend,test_blend_x_xgb, delimiter=",")

[ 0.54393943  0.54340168  0.54486921  0.54463413  0.5455042 ]
4024.2


In [130]:
test_blend_x_xgb[:,:3]


array([[ 0.05068891,  0.47914108,  0.47017003],
       [ 0.00355554,  0.03459964,  0.96184484],
       [ 0.09190651,  0.43008367,  0.47800982],
       ..., 
       [ 0.04040814,  0.38965375,  0.5699381 ],
       [ 0.06617295,  0.5109495 ,  0.42287757],
       [ 0.0445355 ,  0.33600247,  0.61946204]])

In [ ]:
now = datetime.now()
sub_name = '../output/sub_XGB_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

out_df = pd.DataFrame(test_blend_x_xgb[:,:3])
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv(sub_name, index=False)


# ypreds.columns = cols

# df = pd.read_json(open("../input/test.json", "r"))
# ypreds['listing_id'] = df["listing_id"]

# ypreds.to_csv('my_preds.csv', index=None)